In [31]:
import pandas as pd
from ollama import chat
from ollama import ChatResponse

df = pd.read_json("hf://datasets/HuggingFaceH4/MATH-500/test.jsonl", lines=True)

In [32]:
df.head(2)

,problem,solution,answer,subject,level,unique_id
0,"Convert the point $(0,3)$ in rectangular coord...",We have that $r = \sqrt{0^2 + 3^2} = 3.$ Also...,"\left( 3, \frac{\pi}{2} \right)",Precalculus,2,test/precalculus/807.json
1,Define\n\[p = \sum_{k = 1}^\infty \frac{1}{k^2...,We count the number of times $\frac{1}{n^3}$ a...,p - q,Intermediate Algebra,5,test/intermediate_algebra/1994.json


In [33]:
from pydantic import BaseModel
class Answer(BaseModel):
  reasoning: str
  answer: str

class AnswerCorrectness(BaseModel):
  correct:bool



In [37]:
def process_math_problems(df, start_idx=0, end_idx=20):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        response: ChatResponse = chat(model='llama3.1:latest', messages=[
            {
                'role': 'system',
                'content': '''Be a helpful assistant. Your task is to think hard about the problem given.
                You are to reason on the task at hand and output your thinking along with the answer. Your output will be a json with reasoning 
                as on of the key value pair and the other key value pair is going to be answer.''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }],
            format=Answer.model_json_schema()
        )
        answer_content = response['message']['content']
        answer_obj = Answer.model_validate_json(answer_content)
        answer = answer_obj.answer

        response: ChatResponse = chat(model='llama3.1:latest', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], format=AnswerCorrectness.model_json_schema())
        answer_correctness = response['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response'] = answer_content
        df.at[idx, 'llm_answer'] = answer
        df.at[idx, 'is_correct'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [38]:
process_math_problems(df,start_idx=15,end_idx=30)

Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20
Processed row 21
Processed row 22
Processed row 23
Processed row 24
Processed row 25
Processed row 26
Processed row 27
Processed row 28
Processed row 29
Processed row 30


In [39]:
df.to_csv("30 rows.csv")

In [64]:
def generate_training_messages(df, start_idx=0, num_rows=10):
    training_data = []
    end_idx = min(start_idx + num_rows, len(df))
    messages = [{
                'role': 'system',
                'content': '''Be a helpful assistant. Your task is to think hard about the problem given.
                You are to reason on the task at hand and output your thinking along with the answer. Your output will be a json with reasoning 
                as one of the key value pair and the other key value pair is going to be answer.'''
            }]
    for idx in range(start_idx, end_idx):
        row = df.iloc[idx]
        
        # Base messages that are common for all examples
        messages.extend([
            {
                'role': 'user',
                'content': row['problem']
            },
            {
                'role': 'assistant',
                'content': row['llm_raw_response']
            }
        ])
        
        # Add feedback based on correctness
        if not row['is_correct']:
            messages.append({
                'role': 'user',
                'content': f"Let me correct this. The right answer is {row['answer']}. Let's understand the solution: {row['solution']}"
            })
        else:
            messages.append({
                'role': 'user',
                'content': "Good job! Your reasoning and answer are correct!"
            })
        messages.append({
            'role': 'assistant',
            'content': "Understood. I will keep this in mind"
        })
            
       
    
    return messages

In [65]:
training_data = generate_training_messages(df)

In [70]:
training_data

[{'role': 'system',
  'content': 'Be a helpful assistant. Your task is to think hard about the problem given.\n                You are to reason on the task at hand and output your thinking along with the answer. Your output will be a json with reasoning \n                as one of the key value pair and the other key value pair is going to be answer.'},
 {'role': 'user',
  'content': 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$'},
 {'role': 'assistant',
  'content': '{"answer": "(3π/2, 3)" , "reasoning": "To convert a point from rectangular to polar coordinates, we can use the following formulas: r = √(x^2 + y^2) and θ = arctan(y/x). In this case, we have (0, 3), so x = 0 and y = 3. Plugging these values into the formulas, we get r = √(0^2 + 3^2) = √9 = 3 and θ = arctan(3/0). Since arctan is not defined at zero, we need to use a different approach. In this case, we

In [67]:
def test_process_math_problems_te(df, start_idx=11, end_idx=20):
    for idx in range(start_idx, min(len(df), end_idx + 1)):
        messages = training_data + [
            {
                'role': 'system',
                'content': '''Be a helpful assistant. Your task is to think hard about the problem given.
                You are to reason on the task at hand and output your thinking along with the answer. Your output will be a json with reasoning 
                as on of the key value pair and the other key value pair is going to be answer.''',
            },
            {
                'role': 'user',
                'content': df.iloc[idx]['problem'],
            }]
        
        response: ChatResponse = chat(model='llama3.1:latest', messages=messages,
            format=Answer.model_json_schema()
        )
        answer_content = response['message']['content']
        answer_obj = Answer.model_validate_json(answer_content)
        answer = answer_obj.answer

        response: ChatResponse = chat(model='llama3.1:latest', messages=[
            {
                'role': 'system',
                'content': '''You are an intellegent maths professor. I will give you 2 answers. 
                    One Model answer and one student answer. You whether the answer is right or wrong.Return a json with key are correct and value as True or False depeding on your evaluation''',
            },
            {
                'role': 'user',
                'content': f'Model Answer : {df.iloc[idx]["answer"]}, Student Answer : {answer}',
            },
        ], format=AnswerCorrectness.model_json_schema())
        answer_correctness = response['message']['content']
        answer_correctness_obj = AnswerCorrectness.model_validate_json(answer_correctness)
        
        df.at[idx, 'llm_raw_response_test'] = answer_content
        df.at[idx, 'llm_answer_test'] = answer
        df.at[idx, 'is_correct_test'] = answer_correctness_obj.correct
        
        print(f"Processed row {idx}")


In [68]:
test_process_math_problems_te(df)

Processed row 11
Processed row 12
Processed row 13
Processed row 14
Processed row 15
Processed row 16
Processed row 17
Processed row 18
Processed row 19
Processed row 20


In [69]:
df.to_csv('first test.csv')